# Evolution of dynamical systems and sensitivity to initial conditions

In [ ]:
from functools import cache
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.integrate import solve_ivp

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Lotka Volterra (Mathematical Biology Sheet 1 Question 7)

In [ ]:
def lotka_volterra(t, y, rho, b12, b21):
    n1, n2 = y
    n1_dot = n1 * (1 - n1 - b12 * n2)
    n2_dot = rho * n2 * (1 - b21 * n1 - n2)
    return [n1_dot, n2_dot]

In [ ]:
@interact(rho=widgets.FloatSlider(min=0, max=3, value=1, continuous_update=False),
          b12=widgets.FloatSlider(min=0, max=2, value=1.5, continuous_update=False),
          b21=widgets.FloatSlider(min=0, max=2, value=0.5, continuous_update=False))
def lvdemo(rho, b12, b21):
    x_range = [0, 1.5]
    y_range = [0, 1.5]
    tf = 5
    initial_states = [(x, y)
                      for x in np.linspace(*x_range, 9) 
                      for y in np.linspace(*y_range, 9)]
    sols = [solve_ivp(lotka_volterra,
                      [0, tf],
                      i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                      args=(rho, b12, b21))
            for i in initial_states]

    fig = plt.figure(figsize=(8, 8))
    arrowlen = 0.05
    for s in sols:
        t, n1, n2 = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(n1, n2,
                    s=np.linspace(1, 5, len(n1)),
                    c=t,
                    cmap='viridis'
        )

    ax = plt.gca()
    ax.set_xlabel('n1')
    ax.set_ylabel('n2')
    plt.show()

## SIR model

In [ ]:
def sir(t, y, delta):
    susc, inf = y
    return [-susc * inf, susc * inf - delta * inf]

In [ ]:
@interact(delta=widgets.FloatSlider(min=0, max=2, value=1, continuous_update=False))
def sir_demo(delta):
    x_range = [0, 1]
    y_range = [0, 1]
    tf = 10
    initial_states = [(susc, 1 - susc)
                      for susc in np.linspace(*x_range, 21)]
    sols = [solve_ivp(sir, [0, tf], i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                     args=(delta,))
            for i in initial_states]

    fig = plt.figure(figsize=(8, 8))
    arrowlen = 0.05
    for s in sols:
        t, susc, inf = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(susc, inf,
            s=np.linspace(1, 30, len(t)),
            c=t, cmap='viridis'
        )

        dx, dy = sir(0, [susc[0], inf[0]], delta)
        plt.arrow(susc[0] - .5*arrowlen*dx,
                  inf[0] - .5*arrowlen*dy,
                  arrowlen*dx,
                  arrowlen*dy,
                  color='black')

        dx, dy = sir(tf, [susc[-1], inf[-1]], delta)
        plt.arrow(susc[-1] - .5*arrowlen*dx,
                  inf[-1] - .5*arrowlen*dy,
                  arrowlen*dx,
                  arrowlen*dy,
                  color='blue')

    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_range)
    ax.set_ylim(y_range)
    # ax.grid(color='black')
    plt.show()